# Peer-graded Assignment Week 3 Part 1 : Segmenting and Clustering Neighborhoods in Toronto

### A. Import Libraries

In [1]:
!pip install lxml

!pip install html5lib

!pip install BeautifulSoup4

!pip install wikipedia

!pip install geocoder


import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON filesimport numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

     |████████████████████████████████| 5.8MB 29.6MB/s eta 0:00:01
     |████████████████████████████████| 112kB 5.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 102kB 5.2MB/s ta 0:00:011
     |████████████████████████████████| 829kB 31.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    

### B. Parse and Extract data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

wiki_table = soup.body.table.tbody

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

df.head()

df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode       287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


(287, 3)

### C. Drop the cells that have not been assigned and group by postal code

In [11]:
df1=df[df.Borough != "Not assigned"].reset_index(drop=True)
df1.head()



,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [12]:
df2=df1.groupby(["PostalCode"], as_index=False).agg(lambda x: ','.join(set(x)))
df2.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [13]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

In [14]:

df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [15]:
df2.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"



test_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood"])

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df2[df2["PostalCode"]==postcode], ignore_index=True, sort =False)
    
test_df.head(12)

# Peer-graded Assignment Week 3 Part 2-Part 3 : Segmenting and Clustering Neighborhoods in Toronto

### Postal code-Latitude-Longitude

In [37]:
df_geo = pd.read_csv("https://cocl.us/Geospatial_data")
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']

print("Data read into dataframe!") # takes about 30 seconds


Data read into dataframe!


In [38]:

df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Dataframes

In [34]:
df_pos = pd.merge(df2, df_geo, on=['PostalCode'], how='inner')

df_tor = df_pos[['PostalCode','Borough', 'Neighbourhood', 'Latitude', 'Longitude']].copy()

df_tor.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [35]:
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### The Course output 

In [36]:
test_df2 = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood","Latitude","Longitude"])

test_list2 = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list2:
    test_df2 = test_df2.append(df_tor[df_tor["PostalCode"]==postcode], ignore_index=True, sort =False)
    
test_df2.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,DowntownToronto,Central Bay Street,43.657952,-79.387383
1,M2H,NorthYork,Hillcrest Village,43.803762,-79.363452
2,M4B,EastYork,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,EastYork,Leaside,43.709060,-79.363452
5,M4M,EastToronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford,Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"Mount Olive,South Steeles,Thistletown,Silverst...",43.739416,-79.588437
8,M9L,NorthYork,Humber Summit,43.756303,-79.565963
9,M5V,DowntownToronto,"South Niagara,Bathurst Quay,King and Spadina,R...",43.628947,-79.394420
